In [1]:
# Library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import keras
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping



In [2]:
# Standard Normal Variate Function
def snv(input_data):
  
    # Define a new array and populate it with the corrected data  
    output_data = np.zeros_like(input_data)
    for i in range(input_data.shape[0]): 
        # Apply correction
        output_data[i,:] = (input_data[i,:] - np.mean(input_data[i,:])) / np.std(input_data[i,:])
        
    return output_data

In [3]:
# load the dataset
data = pd.read_csv('/content/Acetaminophen_mixture_CNN_Binary.csv', index_col=0)
data = pd.DataFrame(data)
data.drop(['label_name'], axis=1, inplace=True)
data.rename(columns={'Unnamed: 0': 'id'},inplace=True)
y = data['label']
X = data.drop(['label'],axis=1)
X = X.values[:,1:]
X = snv(X)
X = pd.DataFrame(X)
print((X.shape))


(900, 226)


In [4]:
# Train Test Split and reshaping the data
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)
X_train = X_train.values.reshape(X_train.shape[0],X_train.shape[1],1).astype('float32')
X_test= X_test.values.reshape(X_test.shape[0],X_test.shape[1],1).astype('float32')

In [5]:
# Define Model

n_timesteps = X_train.shape[1] 
n_features  = X_train.shape[2]  
model = keras.Sequential(name="model_conv1D")
model.add(keras.layers.Input(shape=(n_timesteps,n_features)))
model.add(keras.layers.Conv1D(filters=64, kernel_size=7, activation='relu', name="Conv1D_1"))
model.add(keras.layers.Conv1D(filters=32, kernel_size=5, activation='relu', name="Conv1D_2"))
model.add(keras.layers.Conv1D(filters=16, kernel_size=2, activation='relu', name="Conv1D_3"))


model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu', name="Dense_1"))
model.add(keras.layers.Dense(n_features, name="Dense_2"))
#model.add(keras.layers.Dense(1, activation='relu'))

model.compile(loss ='mse', optimizer='adam',metrics=['mse','mae'])

In [6]:
model.summary()

Model: "model_conv1D"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1D_1 (Conv1D)            (None, 220, 64)           512       
_________________________________________________________________
Conv1D_2 (Conv1D)            (None, 216, 32)           10272     
_________________________________________________________________
Conv1D_3 (Conv1D)            (None, 215, 16)           1040      
_________________________________________________________________
flatten (Flatten)            (None, 3440)              0         
_________________________________________________________________
Dense_1 (Dense)              (None, 32)                110112    
_________________________________________________________________
Dense_2 (Dense)              (None, 1)                 33        
Total params: 121,969
Trainable params: 121,969
Non-trainable params: 0
________________________________________________

In [7]:
history = model.fit(X_train, y_train, epochs=20,verbose=1)

Epoch 1/20
23/23 [==============================] - 2s 31ms/step - loss: 0.1148 - mse: 0.1148 - mae: 0.2767
Epoch 2/20
23/23 [==============================] - 1s 31ms/step - loss: 0.0139 - mse: 0.0139 - mae: 0.0931
Epoch 3/20
23/23 [==============================] - 1s 31ms/step - loss: 0.0086 - mse: 0.0086 - mae: 0.0731
Epoch 4/20
23/23 [==============================] - 1s 30ms/step - loss: 0.0084 - mse: 0.0084 - mae: 0.0728
Epoch 5/20
23/23 [==============================] - 1s 32ms/step - loss: 0.0060 - mse: 0.0060 - mae: 0.0592
Epoch 6/20
23/23 [==============================] - 1s 32ms/step - loss: 0.0060 - mse: 0.0060 - mae: 0.0595
Epoch 7/20
23/23 [==============================] - 1s 31ms/step - loss: 0.0066 - mse: 0.0066 - mae: 0.0617
Epoch 8/20
23/23 [==============================] - 1s 30ms/step - loss: 0.0046 - mse: 0.0046 - mae: 0.0507
Epoch 9/20
23/23 [==============================] - 1s 31ms/step - loss: 0.0041 - mse: 0.0041 - mae: 0.0471
Epoch 10/20
23/23 [=========

In [8]:
mse = model.evaluate(X_test, y_test)
print('Mean Squared Error!',mse)
print(history.history['mae'])


6/6 [==============================] - 0s 9ms/step - loss: 0.0023 - mse: 0.0023 - mae: 0.0401
Mean Squared Error! [0.0023102008271962404, 0.0023102008271962404, 0.04005202651023865]
[0.27665406465530396, 0.09314193576574326, 0.07305534929037094, 0.0727858766913414, 0.059227123856544495, 0.059511180967092514, 0.061704788357019424, 0.05065327137708664, 0.04710531234741211, 0.05454849824309349, 0.04862481355667114, 0.051486339420080185, 0.047000136226415634, 0.03711612895131111, 0.04232007637619972, 0.04532766342163086, 0.03489181399345398, 0.03859340026974678, 0.03408747538924217, 0.033435020595788956]


In [9]:
#Predicting Mystery Data
myst = pd.read_csv('/content/Mystery_testing_data.csv', index_col=0)
myst = pd.DataFrame(myst)
myst_predictions = pd.DataFrame(myst["label_name"])
myst.drop(['label_name'], axis=1, inplace=True)
# Applying SNV to myst data
myst = myst.values[:,1:]
myst = snv(myst)
myst = pd.DataFrame(myst)

myst= myst.values.reshape(myst.shape[0],myst.shape[1],1).astype('float32')
preds = model.predict(myst)

se = pd.Series(preds.tolist())
myst_predictions.insert(loc=0, column='predictions', value=se.values)
#Saving results
myst_predictions.to_csv('/content/Mystery_Prediction_1DCNN_data_binary.csv')

In [10]:
#Predicting Ternary Samples Data
data = pd.read_csv('/content/ternary_samples.csv', index_col=0)
data = pd.DataFrame(data)
ternary_predictions = pd.DataFrame()

ternary_predictions["label_name"] =data["label_name"]
ternary_predictions['Actual'] = data['label']

data.drop(['label_name'], axis=1, inplace=True)
data.rename(columns={'Unnamed: 0': 'id'},inplace=True)

y = data['label']
X = data.drop(['label'],axis=1)
X = X.values[:,1:]
X = snv(X)
X = pd.DataFrame(X)

ternary= X.values.reshape(X.shape[0],X.shape[1],1).astype('float32')
preds = model.predict(ternary)

se = pd.Series(preds.tolist())
ternary_predictions.insert(loc=0, column='predictions', value=se.values)
#Saving results
ternary_predictions.to_csv('/content/Ternary_Prediction_1DCNN_data_Ternary.csv')

In [20]:
#Predicting Tylenol Data
data = pd.read_csv('tylenol_topcare_samples.csv', index_col=0)
data = pd.DataFrame(data)
ternary_predictions = pd.DataFrame()

ternary_predictions["label_name"] =data["label_name"]


data.drop(['label_name'], axis=1, inplace=True)
data.rename(columns={'Unnamed: 0': 'id'},inplace=True)

X = data.values[:,1:]
X = snv(X)
X = pd.DataFrame(X)
print(X.head(3))

ternary= X.values.reshape(X.shape[0],X.shape[1],1).astype('float32')
preds = model.predict(ternary)

se = pd.Series(preds.tolist())
ternary_predictions.insert(loc=0, column='predictions', value=se.values)
#Saving results
ternary_predictions.to_csv('/content/Tylenol_Prediction_1DCNN.csv')

        0         1         2    ...       223       224       225
0 -1.136921 -1.088035 -1.093519  ...  0.855433  1.031870  0.984094
1 -1.060666 -1.078443 -1.051561  ...  1.461359  1.168942  1.303648
2 -1.073092 -1.038393 -1.032049  ...  1.285524  1.282711  1.157090

[3 rows x 226 columns]
